In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd '/content/drive/MyDrive/Colab Notebooks/EY 2025 DS Challenge'

/content/drive/MyDrive/Colab Notebooks/EY 2025 DS Challenge


# Load Original Dataset

In [3]:
import pandas as pd
import numpy as np
from datetime import timedelta
import geopandas as gpd
from shapely.geometry import Point, Polygon
from scipy.spatial import cKDTree
from xml.etree import ElementTree as ET

In [4]:
# Load data.csv and weather.csv
data_df = pd.read_csv('datasets/data.csv', parse_dates=['datetime'])
data_df.rename(columns={'datetime': 'Datetime'}, inplace=True)
sub_df = pd.read_csv('datasets/template.csv')
footprints_df = pd.read_csv('datasets/footprints2.0.csv')

weather_df = pd.read_csv('datasets/weather.csv', parse_dates=['Datetime'])
gdf_buildings = gpd.read_file("datasets/footprints.kml", driver='KML')

display(data_df.head())
display(sub_df.head())
display(weather_df.head())
display(gdf_buildings.head())

<ipython-input-4-8b5525efab95>:2: UserWarning: Parsing dates in %d-%m-%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  data_df = pd.read_csv('datasets/data.csv', parse_dates=['datetime'])
<ipython-input-4-8b5525efab95>:7: UserWarning: Parsing dates in %d/%m/%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  weather_df = pd.read_csv('datasets/weather.csv', parse_dates=['Datetime'])


,Longitude,Latitude,Datetime,UHI Index
0,-73.909167,40.813107,2021-07-24 15:53:00,1.030289
1,-73.909187,40.813045,2021-07-24 15:53:00,1.030289
2,-73.909215,40.812978,2021-07-24 15:53:00,1.023798
3,-73.909242,40.812908,2021-07-24 15:53:00,1.023798
4,-73.909257,40.812845,2021-07-24 15:53:00,1.021634


,Longitude,Latitude,UHI Index
0,-73.971665,40.788763,NaN
1,-73.971928,40.788875,NaN
2,-73.967080,40.789080,NaN
3,-73.972550,40.789082,NaN
4,-73.969697,40.787953,NaN


,Region,Longitude,Latitude,Datetime,Air Temp at Surface,Relative Humidity,Avg Wind Speed,Wind Direction,Solar Flux
0,Bronx,-73.89352,40.87248,2021-07-24 06:00:00,19.3,88.2,0.8,335,12
1,Bronx,-73.89352,40.87248,2021-07-24 06:05:00,19.4,87.9,0.8,329,18
2,Bronx,-73.89352,40.87248,2021-07-24 06:10:00,19.3,87.6,0.7,321,25
3,Bronx,-73.89352,40.87248,2021-07-24 06:15:00,19.4,87.4,0.5,307,33
4,Bronx,-73.89352,40.87248,2021-07-24 06:20:00,19.4,87.0,0.2,301,42


,Name,Description,geometry
0,,,"MULTIPOLYGON (((-73.91903 40.8482, -73.91933 4..."
1,,,"MULTIPOLYGON (((-73.92195 40.84963, -73.92191 ..."
2,,,"MULTIPOLYGON (((-73.9205 40.85011, -73.92045 4..."
3,,,"MULTIPOLYGON (((-73.92056 40.8514, -73.92053 4..."
4,,,"MULTIPOLYGON (((-73.91234 40.85218, -73.91247 ..."


# Assign a region (Manhattan / Bronx) to train and submission data

In [5]:
# Function to parse KML and extract county boundaries
def parse_kml(kml_file):
    tree = ET.parse(kml_file)
    root = tree.getroot()

    # Extract all polygons and their associated county names
    polygons = []
    county_names = []

    # Loop through each <Placemark> element in the KML file
    for placemark in root.iter('{http://www.opengis.net/kml/2.2}Placemark'):
        county_name = None
        polygon_coords = []

        # Extract county name (from <Data name="name"><value>...</value></Data>)
        for data in placemark.iter('{http://www.opengis.net/kml/2.2}Data'):
            if data.attrib.get('name') == 'name':
                county_name = data.find('{http://www.opengis.net/kml/2.2}value').text

        # Extract polygon coordinates
        for polygon in placemark.iter('{http://www.opengis.net/kml/2.2}Polygon'):
            for coords in polygon.iter('{http://www.opengis.net/kml/2.2}coordinates'):
                coords_text = coords.text.strip()
                for coord in coords_text.split():
                    # Try to unpack coordinates, ignore altitude if present
                    parts = coord.split(',')
                    if len(parts) >= 2:
                        lon, lat = map(float, parts[:2])
                        polygon_coords.append((lon, lat))

        # Convert coordinates into a Polygon geometry
        if polygon_coords and county_name:
            polygons.append(Polygon(polygon_coords))
            county_names.append(county_name)

    # Create a GeoDataFrame with county boundaries
    county_gdf = gpd.GeoDataFrame({
        'CountyName': county_names,
        'geometry': polygons
    }, crs="EPSG:4326")

    return county_gdf

# Function to find county for each coordinate
def get_county_for_coordinates(df, boundary_gdf):
    # Convert the Latitude and Longitude into a GeoSeries of Points
    geometry = [Point(lon, lat) for lon, lat in zip(df['Longitude'], df['Latitude'])]
    points_gdf = gpd.GeoDataFrame(df, geometry=geometry, crs="EPSG:4326")

    # Initialize a list to store county names
    county_names = []

    # Check which county the point belongs to
    for _, point in points_gdf.iterrows():
        county = None
        for _, county_row in boundary_gdf.iterrows():
            if point['geometry'].within(county_row['geometry']):
                county = county_row['CountyName']  # Get the county name
                break
        county_names.append(county)

    # Add the county names as a new column to the dataframe
    df['County'] = county_names
    return df

ny_boundary = parse_kml("datasets/boundaries.kml")

# Apply the function to data_df and sub_df
data_df = get_county_for_coordinates(data_df, ny_boundary)
sub_df = get_county_for_coordinates(sub_df, ny_boundary)

In [6]:
# Step 1: Create a new column 'Region' based on 'County' column
def assign_region(county_name):
    if county_name == 'New York County':
        return 'Manhattan'
    elif county_name == 'Bronx County':
        return 'Bronx'
    else:
        return 'Others'

# Apply the function to both dataframes
data_df['Region'] = data_df['County'].apply(assign_region)
sub_df['Region'] = sub_df['County'].apply(assign_region)

# # Step 2: Drop the 'County' column
data_df.drop('County', axis=1, inplace=True)
sub_df.drop('County', axis=1, inplace=True)

# Step 3: Count the number of "Others" regions
others_count_data = data_df['Region'].value_counts().get('Others', 0)
others_count_sub = sub_df['Region'].value_counts().get('Others', 0)

# Output the counts
print(f"Number of 'Others' regions in data_df: {others_count_data}")
print(f"Number of 'Others' regions in sub_df: {others_count_sub}")


Number of 'Others' regions in data_df: 0
Number of 'Others' regions in sub_df: 0


In [7]:
display(data_df.head())
display(sub_df.head())

,Longitude,Latitude,Datetime,UHI Index,Region
0,-73.909167,40.813107,2021-07-24 15:53:00,1.030289,Bronx
1,-73.909187,40.813045,2021-07-24 15:53:00,1.030289,Bronx
2,-73.909215,40.812978,2021-07-24 15:53:00,1.023798,Bronx
3,-73.909242,40.812908,2021-07-24 15:53:00,1.023798,Bronx
4,-73.909257,40.812845,2021-07-24 15:53:00,1.021634,Bronx


,Longitude,Latitude,UHI Index,Region
0,-73.971665,40.788763,NaN,Manhattan
1,-73.971928,40.788875,NaN,Manhattan
2,-73.967080,40.789080,NaN,Manhattan
3,-73.972550,40.789082,NaN,Manhattan
4,-73.969697,40.787953,NaN,Manhattan


# Map footprints onto orignal data and submission data
[To do: try setting different buffersize for each region]

In [8]:
# Set a buffer size variable for radius (in meters)
# Manhattan is around 59.1km big, and bronx is around 110km big
# buffer_sizes = {
#     'Manhattan': 1500,  # Buffer size for Manhattan in meters
#     'Bronx': 1800,      # Buffer size for Bronx in meters
# }

buffer_size = 1500

In [9]:
# Convert Longitude and Latitude columns into Point geometries
train_gdf = gpd.GeoDataFrame(data_df, geometry=gpd.points_from_xy(data_df['Longitude'], data_df['Latitude']))
sub_gdf = gpd.GeoDataFrame(sub_df, geometry=gpd.points_from_xy(sub_df['Longitude'], sub_df['Latitude']))

# Ensure all dataframes use the same CRS (Coordinate Reference System)
train_gdf.set_crs('EPSG:4326', allow_override=True, inplace=True)
sub_gdf.set_crs('EPSG:4326', allow_override=True, inplace=True)
gdf_buildings.set_crs('EPSG:4326', allow_override=True, inplace=True)

display(train_gdf.head())
display(sub_gdf.head())

,Longitude,Latitude,Datetime,UHI Index,Region,geometry
0,-73.909167,40.813107,2021-07-24 15:53:00,1.030289,Bronx,POINT (-73.90917 40.81311)
1,-73.909187,40.813045,2021-07-24 15:53:00,1.030289,Bronx,POINT (-73.90919 40.81304)
2,-73.909215,40.812978,2021-07-24 15:53:00,1.023798,Bronx,POINT (-73.90922 40.81298)
3,-73.909242,40.812908,2021-07-24 15:53:00,1.023798,Bronx,POINT (-73.90924 40.81291)
4,-73.909257,40.812845,2021-07-24 15:53:00,1.021634,Bronx,POINT (-73.90926 40.81284)


,Longitude,Latitude,UHI Index,Region,geometry
0,-73.971665,40.788763,NaN,Manhattan,POINT (-73.97166 40.78876)
1,-73.971928,40.788875,NaN,Manhattan,POINT (-73.97193 40.78888)
2,-73.967080,40.789080,NaN,Manhattan,POINT (-73.96708 40.78908)
3,-73.972550,40.789082,NaN,Manhattan,POINT (-73.97255 40.78908)
4,-73.969697,40.787953,NaN,Manhattan,POINT (-73.9697 40.78795)


## Nearest Building Distance

In [10]:
from sklearn.neighbors import BallTree
# # Function to calculate nearest building distance for each point using vectorized operations
# def nearest_building_distance(sub_gdf, gdf_buildings):
#     # Convert building footprints and sub points into the same CRS if not already
#     gdf_buildings = gdf_buildings.to_crs(sub_gdf.crs)

#     # Assign buffer size based on 'Region' column in the sub_gdf
#     sub_gdf['buffer'] = sub_gdf['Region'].map(buffer_sizes)

#     # Create a buffer around each sub point (this is a vectorized operation)
#     sub_gdf['buffer_geometry'] = sub_gdf.geometry.buffer(sub_gdf['buffer'])

#     # Spatial join to find nearest buildings within the buffer
#     nearest_buildings = gpd.sjoin_nearest(sub_gdf, gdf_buildings, how="left", distance_col="nearest_building_distance")

#     return nearest_buildings['nearest_building_distance']

# # Apply the optimized function to calculate nearest building distance for the sub data
# sub_df['nearest_building_distance'] = nearest_building_distance(sub_gdf, gdf_buildings)

# # Apply the same to train data if necessary (can be done in a similar manner)
# data_df['nearest_building_distance'] = nearest_building_distance(train_gdf, gdf_buildings)

# Function to calculate the nearest building distance using BallTree (vectorized operations)
def nearest_building_distance_balltree(sub_gdf, gdf_buildings, buffer_size):
    # Convert building footprints and sub points into numpy arrays of coordinates (lon, lat)
    building_coords = np.array([(geom.centroid.x, geom.centroid.y) for geom in gdf_buildings.geometry])
    sub_coords = np.array([(geom.x, geom.y) for geom in sub_gdf.geometry])

    # Initialize BallTree with geospatial coordinates (convert to radians for haversine)
    tree = BallTree(np.radians(building_coords), metric='haversine')

    # Convert sub points to radians
    sub_coords_radians = np.radians(sub_coords)

    # Query BallTree to find the nearest building
    dist, _ = tree.query(sub_coords_radians, k=1)  # k=1 means nearest neighbor

    # Convert the distance from radians back to meters (radius of Earth = 6371000 meters)
    dist_in_meters = dist * 6371000

    return dist_in_meters.flatten()

# Apply the function to calculate the nearest building distance for the sub data
sub_df['nearest_building_distance'] = nearest_building_distance_balltree(sub_gdf, gdf_buildings, buffer_size)

# Apply the same to train data if necessary
data_df['nearest_building_distance'] = nearest_building_distance_balltree(train_gdf, gdf_buildings, buffer_size)

# Check the first few rows to confirm the new column
display(sub_df.head())
display(data_df.head())



,Longitude,Latitude,UHI Index,Region,nearest_building_distance
0,-73.971665,40.788763,NaN,Manhattan,24.223804
1,-73.971928,40.788875,NaN,Manhattan,18.437818
2,-73.967080,40.789080,NaN,Manhattan,45.864314
3,-73.972550,40.789082,NaN,Manhattan,4.973082
4,-73.969697,40.787953,NaN,Manhattan,39.096254


,Longitude,Latitude,Datetime,UHI Index,Region,nearest_building_distance
0,-73.909167,40.813107,2021-07-24 15:53:00,1.030289,Bronx,28.702934
1,-73.909187,40.813045,2021-07-24 15:53:00,1.030289,Bronx,27.747556
2,-73.909215,40.812978,2021-07-24 15:53:00,1.023798,Bronx,26.465094
3,-73.909242,40.812908,2021-07-24 15:53:00,1.023798,Bronx,25.873093
4,-73.909257,40.812845,2021-07-24 15:53:00,1.021634,Bronx,26.359730


## Building Density

In [11]:
# # Convert building geometries to numpy array (lon, lat) from centroids
# building_coords = np.array([(geom.x, geom.y) for geom in gdf_buildings.geometry.centroid])

# # Initialize BallTree with geospatial coordinates (convert to radians for haversine)
# tree = BallTree(np.radians(building_coords), metric='haversine')

# # Function to calculate building density (number of nearby buildings within a radius)
# def building_density_balltree(row, tree):
#     # Check the region and assign the appropriate buffer size (in meters)
#     if row['Region'] == 'Manhattan':
#         radius = buffer_sizes['Manhattan']  # Use Manhattan's buffer size (in meters)
#     elif row['Region'] == 'Bronx':
#         radius = buffer_sizes['Bronx']  # Use Bronx's buffer size (in meters)
#     else:
#         return 0  # If the region is not Manhattan or Bronx, return 0 for density

#     # Convert the coordinates of the row's point to radians
#     point = np.radians([[row.geometry.x, row.geometry.y]])  # Convert point to radians

#     # Query the BallTree for buildings within the specified radius (converted to radians)
#     indices = tree.query_radius(point, r=radius / 6371000)  # Radius in kilometers (divide by Earth's radius in meters)

#     # Return the number of nearby buildings
#     return len(indices[0])

# # Apply BallTree density calculation for the sub and train datasets
# train_gdf['building_density'] = train_gdf.apply(building_density_balltree, axis=1, tree=tree)
# sub_gdf['building_density'] = sub_gdf.apply(building_density_balltree, axis=1, tree=tree)

# # Add the 'building_density' to the dataframes (if needed)
# data_df['building_density'] = train_gdf['building_density']
# sub_df['building_density'] = sub_gdf['building_density']


from shapely import wkt

# Convert 'footprints_df' to GeoDataFrame with 'the_geom' as geometry
footprints_df['geometry'] = footprints_df['the_geom'].apply(wkt.loads)  # Parse WKT strings into geometries
gdf_buildings = gpd.GeoDataFrame(footprints_df, geometry='geometry')

# Ensure the GeoDataFrame has a CRS (Coordinate Reference System)
gdf_buildings.set_crs('EPSG:4326', allow_override=True, inplace=True)

# Extract the building coordinates from the centroids of the geometries
building_coords = np.array([(geom.centroid.x, geom.centroid.y) for geom in gdf_buildings.geometry])

# Initialize BallTree with geospatial coordinates (convert to radians for haversine)
tree = BallTree(np.radians(building_coords), metric='haversine')

# Function to calculate building density (number of nearby buildings within a radius)
def building_density_balltree(row, tree, r=1500):
    point = np.radians([[row['Longitude'], row['Latitude']]])  # Convert point to radians
    indices = tree.query_radius(point, r / 6371000)  # Radius in kilometers, convert to radians
    return len(indices[0])  # Return the number of nearby buildings

# Apply BallTree density calculation for data_df (based on Longitude and Latitude)
data_df['building_density'] = data_df.apply(building_density_balltree, axis=1, tree=tree)
sub_df['building_density'] = sub_df.apply(building_density_balltree, axis=1, tree=tree)


## Building Age

In [12]:
# Define the current year
current_year = 2021

# Ensure CNSTRCT_YR column is numeric (handle missing or invalid values)
gdf_buildings['CNSTRCT_YR'] = pd.to_numeric(gdf_buildings['CNSTRCT_YR'], errors='coerce')

# Calculate Building Age (ignore missing values)
gdf_buildings['Building_Age'] = current_year - gdf_buildings['CNSTRCT_YR']
gdf_buildings = gdf_buildings.dropna(subset=['Building_Age'])  # Remove NaNs

# Convert building footprints to numpy array (lon, lat) and extract ages
building_coords = np.array([(geom.centroid.x, geom.centroid.y) for geom in gdf_buildings.geometry])
building_ages = gdf_buildings['Building_Age'].values  # Extract building ages

# Initialize BallTree (convert coordinates to radians for haversine distance)
tree = BallTree(np.radians(building_coords), metric='haversine')

# Function to calculate the average building age within the buffer size
def average_building_age(df, tree, buffer_size, building_ages):
    coords = np.array([(geom.x, geom.y) for geom in df.geometry])  # Extract point coordinates
    coords_radians = np.radians(coords)  # Convert to radians

    # Query buildings within the buffer (convert buffer to radians)
    indices = tree.query_radius(coords_radians, buffer_size / 6371000)

    # Compute the average age of nearby buildings
    avg_ages = [
        np.mean(building_ages[idx]) if len(idx) > 0 else np.nan  # Compute average, handle empty cases
        for idx in indices
    ]

    return avg_ages

# Apply function to calculate average building age for both datasets
sub_df['avg_building_age'] = average_building_age(sub_gdf, tree, buffer_size, building_ages)
data_df['avg_building_age'] = average_building_age(train_gdf, tree, buffer_size, building_ages)

# Check the output
display(data_df[['Longitude', 'Latitude', 'avg_building_age']].head())
display(sub_df[['Longitude', 'Latitude', 'avg_building_age']].head())


,Longitude,Latitude,avg_building_age
0,-73.909167,40.813107,76.409679
1,-73.909187,40.813045,76.425754
2,-73.909215,40.812978,76.404365
3,-73.909242,40.812908,76.447440
4,-73.909257,40.812845,76.461466


,Longitude,Latitude,avg_building_age
0,-73.971665,40.788763,106.813675
1,-73.971928,40.788875,106.924788
2,-73.967080,40.789080,106.268013
3,-73.972550,40.789082,107.083811
4,-73.969697,40.787953,106.378912


## Building Height and Elevation

In [13]:
# Ensure numeric values for calculations
gdf_buildings['GROUNDELEV'] = pd.to_numeric(gdf_buildings['GROUNDELEV'], errors='coerce')
gdf_buildings['HEIGHTROOF'] = pd.to_numeric(gdf_buildings['HEIGHTROOF'], errors='coerce')

# Calculate Height Above Ground
gdf_buildings['Height_Above_Ground'] = gdf_buildings['HEIGHTROOF'] - gdf_buildings['GROUNDELEV']

# Remove NaN values (to avoid errors in computations)
gdf_buildings = gdf_buildings.dropna(subset=['GROUNDELEV', 'HEIGHTROOF', 'Height_Above_Ground'])

# Convert building footprints to numpy array (lon, lat)
building_coords = np.array([(geom.centroid.x, geom.centroid.y) for geom in gdf_buildings.geometry])
building_elevations = gdf_buildings['GROUNDELEV'].values  # Elevation
building_heights = gdf_buildings['HEIGHTROOF'].values  # Total Height
height_above_ground = gdf_buildings['Height_Above_Ground'].values  # Height above ground

# Initialize BallTree (convert coordinates to radians)
tree = BallTree(np.radians(building_coords), metric='haversine')

# Function to calculate average values using BallTree
def average_building_metrics(df, tree, buffer_size, values):
    coords = np.array([(geom.x, geom.y) for geom in df.geometry])
    coords_radians = np.radians(coords)

    # Query buildings within buffer
    indices = tree.query_radius(coords_radians, buffer_size / 6371000)

    # Compute average for each metric
    avg_values = [
        np.mean(values[idx]) if len(idx) > 0 else np.nan  # Avoid errors when no nearby buildings
        for idx in indices
    ]
    return avg_values

# Apply function to compute average building metrics
sub_df['avg_building_elevation'] = average_building_metrics(sub_gdf, tree, buffer_size, building_elevations)
sub_df['avg_building_height'] = average_building_metrics(sub_gdf, tree, buffer_size, building_heights)
sub_df['avg_height_above_ground'] = average_building_metrics(sub_gdf, tree, buffer_size, height_above_ground)

data_df['avg_building_elevation'] = average_building_metrics(train_gdf, tree, buffer_size, building_elevations)
data_df['avg_building_height'] = average_building_metrics(train_gdf, tree, buffer_size, building_heights)
data_df['avg_height_above_ground'] = average_building_metrics(train_gdf, tree, buffer_size, height_above_ground)

# Check results
display(sub_df[['Longitude', 'Latitude', 'avg_building_elevation', 'avg_building_height', 'avg_height_above_ground']].head())
display(data_df[['Longitude', 'Latitude', 'avg_building_elevation', 'avg_building_height', 'avg_height_above_ground']].head())

,Longitude,Latitude,avg_building_elevation,avg_building_height,avg_height_above_ground
0,-73.971665,40.788763,72.826956,95.329320,22.502364
1,-73.971928,40.788875,72.876599,95.641470,22.764872
2,-73.967080,40.789080,70.930495,88.069186,17.138691
3,-73.972550,40.789082,72.799308,96.101098,23.301791
4,-73.969697,40.787953,72.361802,92.955090,20.593288


,Longitude,Latitude,avg_building_elevation,avg_building_height,avg_height_above_ground
0,-73.909167,40.813107,59.732250,39.734542,-19.997708
1,-73.909187,40.813045,59.670272,39.761382,-19.908890
2,-73.909215,40.812978,59.600471,39.763419,-19.837052
3,-73.909242,40.812908,59.550175,39.775245,-19.774930
4,-73.909257,40.812845,59.513618,39.775971,-19.737647


## Final Look after combining Footprints

In [14]:
display(data_df)
display(sub_df)

,Longitude,Latitude,Datetime,UHI Index,Region,nearest_building_distance,building_density,avg_building_age,avg_building_elevation,avg_building_height,avg_height_above_ground
0,-73.909167,40.813107,2021-07-24 15:53:00,1.030289,Bronx,28.702934,10957,76.409679,59.732250,39.734542,-19.997708
1,-73.909187,40.813045,2021-07-24 15:53:00,1.030289,Bronx,27.747556,10942,76.425754,59.670272,39.761382,-19.908890
2,-73.909215,40.812978,2021-07-24 15:53:00,1.023798,Bronx,26.465094,10923,76.404365,59.600471,39.763419,-19.837052
3,-73.909242,40.812908,2021-07-24 15:53:00,1.023798,Bronx,25.873093,10901,76.447440,59.550175,39.775245,-19.774930
4,-73.909257,40.812845,2021-07-24 15:53:00,1.021634,Bronx,26.359730,10883,76.461466,59.513618,39.775971,-19.737647
...,...,...,...,...,...,...,...,...,...,...,...
11224,-73.957050,40.790333,2021-07-24 15:57:00,0.972470,Manhattan,161.437295,12349,101.955419,58.208364,73.181264,14.972900
11225,-73.957063,40.790308,2021-07-24 15:57:00,0.972470,Manhattan,161.166533,12344,101.963042,58.221419,73.191028,14.969610
11226,-73.957093,40.790270,2021-07-24 15:57:00,0.981124,Manhattan,161.138005,12335,101.964168,58.224196,73.228970,15.004774
11227,-73.957112,40.790253,2021-07-24 15:59:00,0.981245,Manhattan,161.353627,12336,101.966226,58.248520,73.240985,14.992465


,Longitude,Latitude,UHI Index,Region,nearest_building_distance,building_density,avg_building_age,avg_building_elevation,avg_building_height,avg_height_above_ground
0,-73.971665,40.788763,NaN,Manhattan,24.223804,9480,106.813675,72.826956,95.329320,22.502364
1,-73.971928,40.788875,NaN,Manhattan,18.437818,9425,106.924788,72.876599,95.641470,22.764872
2,-73.967080,40.789080,NaN,Manhattan,45.864314,10229,106.268013,70.930495,88.069186,17.138691
3,-73.972550,40.789082,NaN,Manhattan,4.973082,9363,107.083811,72.799308,96.101098,23.301791
4,-73.969697,40.787953,NaN,Manhattan,39.096254,9745,106.378912,72.361802,92.955090,20.593288
...,...,...,...,...,...,...,...,...,...,...
1035,-73.919388,40.813803,NaN,Bronx,27.991765,8911,76.912668,56.716524,41.501136,-15.215388
1036,-73.931033,40.833178,NaN,Bronx,25.217523,8922,91.454107,71.470845,52.338347,-19.132499
1037,-73.934647,40.854542,NaN,Manhattan,19.660984,5461,93.230841,98.281975,55.533750,-42.748225
1038,-73.917223,40.815413,NaN,Bronx,34.562437,9502,76.978893,57.478799,41.349281,-16.129518


# Map weather data onto the original and submission data
Here we assume that the submission data is collected around the same time with the orginal data provided.

## Map the closest datetime to the submission data based on the coordinates with original data
If a coordinate is found to have similar time, it takes the closest time in 10m radius

In [15]:
distance_threshold = 10

In [16]:
from sklearn.neighbors import KDTree
import datetime

# Function to calculate the midpoint of two times
def calculate_midpoint_time(time1, time2):
    time1 = pd.to_datetime(time1)
    time2 = pd.to_datetime(time2)
    midpoint = time1 + (time2 - time1) / 2
    return midpoint

# Function to match closest coordinates and estimate time
def match_closest_coordinates_and_estimate_time(train_df, sub_df):
    # Convert latitude and longitude to a 2D array for KDTree
    train_coords = train_df[['Latitude', 'Longitude']].values
    sub_coords = sub_df[['Latitude', 'Longitude']].values

    # Create a KDTree from the training data
    tree = KDTree(train_coords)

    # Initialize a list to store estimated times
    estimated_times = []

    # Iterate over each sub record
    for sub_row in sub_df.itertuples():
        sub_coord = (sub_row.Latitude, sub_row.Longitude)

        # Find the closest training coordinates using KDTree
        dist, ind = tree.query([sub_coord], k=2)  # k=2 to find the two closest matches

        # Get the closest and second closest index
        closest_index = ind[0][0]
        second_closest_index = ind[0][1]

        # If both coordinates are very close, calculate the midpoint time
        if dist[0][0] < distance_threshold:  # [edit this if needed]
            time1 = train_df.loc[closest_index, 'Datetime']
            time2 = train_df.loc[second_closest_index, 'Datetime']
            estimated_time = calculate_midpoint_time(time1, time2)
        else:
            # If the closest coordinates are far enough apart, use the time from the closest match
            estimated_time = train_df.loc[closest_index, 'Datetime']

        # Append the estimated time
        estimated_times.append(estimated_time)

    # Add the estimated times as a new column in the sub DataFrame
    sub_df['Datetime'] = estimated_times

    return sub_df

# Call the function to match coordinates and estimate time
sub_df = match_closest_coordinates_and_estimate_time(data_df, sub_df)


## Based on the region and time, assign the closest weather

In [17]:
from geopy.distance import geodesic

# Filter weather data for each region (Manhattan and Bronx)
manhattan_weather = weather_df[weather_df['Region'] == 'Manhattan']
bronx_weather = weather_df[weather_df['Region'] == 'Bronx']

# Define a function to find the closest datetime in weather data for a given row in data_df
def find_closest_weather_datetime(row, weather_df):
    data_datetime = pd.to_datetime(row['Datetime'])

    # Find the closest datetime in weather data
    weather_df['Datetime'] = pd.to_datetime(weather_df['Datetime'])
    closest_row = weather_df.iloc[(weather_df['Datetime'] - data_datetime).abs().argmin()]

    return closest_row

# Function to assign weather data to data_df based on Region and closest Datetime
def assign_weather_data(row):
    if row['Region'] == 'Manhattan':
        weather_row = find_closest_weather_datetime(row, manhattan_weather)
    elif row['Region'] == 'Bronx':
        weather_row = find_closest_weather_datetime(row, bronx_weather)
    else:
        return pd.Series([None] * 5, index=['Air Temp at Surface', 'Relative Humidity', 'Avg Wind Speed', 'Wind Direction', 'Solar Flux'])

    # Return the weather data as a new row
    return pd.Series({
        'Air Temp at Surface': weather_row['Air Temp at Surface'],
        'Relative Humidity': weather_row['Relative Humidity'],
        'Avg Wind Speed': weather_row['Avg Wind Speed'],
        'Wind Direction': weather_row['Wind Direction'],
        'Solar Flux': weather_row['Solar Flux']
    })

weather_data = data_df.apply(assign_weather_data, axis=1)
data_df = pd.concat([data_df, weather_data], axis=1)

weather_data = sub_df.apply(assign_weather_data, axis=1)
sub_df = pd.concat([sub_df, weather_data], axis=1)

## Final look on the combined weather info onto the original and submission datasets

In [18]:
display(data_df)
display(sub_df)

,Longitude,Latitude,Datetime,UHI Index,Region,nearest_building_distance,building_density,avg_building_age,avg_building_elevation,avg_building_height,avg_height_above_ground,Air Temp at Surface,Relative Humidity,Avg Wind Speed,Wind Direction,Solar Flux
0,-73.909167,40.813107,2021-07-24 15:53:00,1.030289,Bronx,28.702934,10957,76.409679,59.732250,39.734542,-19.997708,27.2,47.3,2.6,165.0,621.0
1,-73.909187,40.813045,2021-07-24 15:53:00,1.030289,Bronx,27.747556,10942,76.425754,59.670272,39.761382,-19.908890,27.2,47.3,2.6,165.0,621.0
2,-73.909215,40.812978,2021-07-24 15:53:00,1.023798,Bronx,26.465094,10923,76.404365,59.600471,39.763419,-19.837052,27.2,47.3,2.6,165.0,621.0
3,-73.909242,40.812908,2021-07-24 15:53:00,1.023798,Bronx,25.873093,10901,76.447440,59.550175,39.775245,-19.774930,27.2,47.3,2.6,165.0,621.0
4,-73.909257,40.812845,2021-07-24 15:53:00,1.021634,Bronx,26.359730,10883,76.461466,59.513618,39.775971,-19.737647,27.2,47.3,2.6,165.0,621.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11224,-73.957050,40.790333,2021-07-24 15:57:00,0.972470,Manhattan,161.437295,12349,101.955419,58.208364,73.181264,14.972900,26.8,46.7,3.4,196.0,605.0
11225,-73.957063,40.790308,2021-07-24 15:57:00,0.972470,Manhattan,161.166533,12344,101.963042,58.221419,73.191028,14.969610,26.8,46.7,3.4,196.0,605.0
11226,-73.957093,40.790270,2021-07-24 15:57:00,0.981124,Manhattan,161.138005,12335,101.964168,58.224196,73.228970,15.004774,26.8,46.7,3.4,196.0,605.0
11227,-73.957112,40.790253,2021-07-24 15:59:00,0.981245,Manhattan,161.353627,12336,101.966226,58.248520,73.240985,14.992465,27.0,46.1,2.7,209.0,620.0


,Longitude,Latitude,UHI Index,Region,nearest_building_distance,building_density,avg_building_age,avg_building_elevation,avg_building_height,avg_height_above_ground,Datetime,Air Temp at Surface,Relative Humidity,Avg Wind Speed,Wind Direction,Solar Flux
0,-73.971665,40.788763,NaN,Manhattan,24.223804,9480,106.813675,72.826956,95.329320,22.502364,2021-07-24 15:30:00,27.3,45.4,3.8,202.0,349.0
1,-73.971928,40.788875,NaN,Manhattan,18.437818,9425,106.924788,72.876599,95.641470,22.764872,2021-07-24 15:30:00,27.3,45.4,3.8,202.0,349.0
2,-73.967080,40.789080,NaN,Manhattan,45.864314,10229,106.268013,70.930495,88.069186,17.138691,2021-07-24 15:29:00,27.3,45.4,3.8,202.0,349.0
3,-73.972550,40.789082,NaN,Manhattan,4.973082,9363,107.083811,72.799308,96.101098,23.301791,2021-07-24 15:30:00,27.3,45.4,3.8,202.0,349.0
4,-73.969697,40.787953,NaN,Manhattan,39.096254,9745,106.378912,72.361802,92.955090,20.593288,2021-07-24 15:29:00,27.3,45.4,3.8,202.0,349.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1035,-73.919388,40.813803,NaN,Bronx,27.991765,8911,76.912668,56.716524,41.501136,-15.215388,2021-07-24 15:26:00,27.3,44.4,3.7,162.0,511.0
1036,-73.931033,40.833178,NaN,Bronx,25.217523,8922,91.454107,71.470845,52.338347,-19.132499,2021-07-24 15:35:00,26.9,47.7,3.5,149.0,292.0
1037,-73.934647,40.854542,NaN,Manhattan,19.660984,5461,93.230841,98.281975,55.533750,-42.748225,2021-07-24 15:32:00,27.3,45.4,3.8,202.0,349.0
1038,-73.917223,40.815413,NaN,Bronx,34.562437,9502,76.978893,57.478799,41.349281,-16.129518,2021-07-24 15:28:00,27.1,47.3,4.5,170.0,563.0


# Combine Sentinel2 Information

In [19]:
!pip install planetary-computer rioxarray

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.9/61.9 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.2/194.2 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 26.4 MB/s eta 0:00:00


In [20]:
# Supress Warnings
import warnings
warnings.filterwarnings('ignore')

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Multi-dimensional arrays and datasets
import xarray as xr

# Geospatial raster data handling
import rioxarray as rxr

# Geospatial data analysis
import geopandas as gpd

# Geospatial operations
import rasterio
from rasterio import windows
from rasterio import features
from rasterio import warp
from rasterio.warp import transform_bounds
from rasterio.windows import from_bounds

# Image Processing
from PIL import Image

# Coordinate transformations
from pyproj import Proj, Transformer, CRS

# Planetary Computer Tools
import pystac_client
import planetary_computer as pc
from pystac.extensions.eo import EOExtension as eo

# Others
import os
from tqdm import tqdm

## Load S2 Tiff

In [21]:
# Open the GeoTIFF file
S2_TIFF = "datasets/S2_sample_all.tiff"

# Read the bands from the GeoTIFF file
with rasterio.open(S2_TIFF) as src1:
    sband1 = src1.read(1)  # Band [B01]
    sband2 = src1.read(2)  # Band [B02]
    sband3 = src1.read(3)  # Band [B03]
    sband4 = src1.read(4)  # Band [B04]
    sband5 = src1.read(5)  # Band [B05]
    sband6 = src1.read(6)  # Band [B06]
    sband7 = src1.read(7)  # Band [B07]
    sband8 = src1.read(8)  # Band [B08]
    sband8a = src1.read(9)  # Band [B8a]
    sband11 = src1.read(10)  # Band [B11]
    sband12 = src1.read(11)  # Band [B12]

## Extract S2 Info

In [22]:
# Extracts satellite band values from a GeoTIFF based on coordinates from a csv file and returns them in a DataFrame.

def map_satellite_data(tiff_path, csv_path):

    # Load the GeoTIFF data
    data = rxr.open_rasterio(tiff_path)
    tiff_crs = data.rio.crs

    # Read the Excel file using pandas
    df = pd.read_csv(csv_path)
    latitudes = df['Latitude'].values
    longitudes = df['Longitude'].values

    # 3. Convert lat/long to the GeoTIFF's CRS
    # Create a Proj object for EPSG:4326 (WGS84 - lat/long) and the GeoTIFF's CRS
    proj_wgs84 = Proj(init='epsg:4326')  # EPSG:4326 is the common lat/long CRS
    proj_tiff = Proj(tiff_crs)

    # Create a transformer object
    transformer = Transformer.from_proj(proj_wgs84, proj_tiff)

    B01_values = []
    B02_values = []
    B03_values = []
    B04_values = []
    B05_values = []
    B06_values = []
    B07_values = []
    B08_values = []
    B8A_values = []
    B11_values = []
    B12_values = []

# Iterate over the latitudes and longitudes, and extract the corresponding band values
    for lat, lon in tqdm(zip(latitudes, longitudes), total=len(latitudes), desc="Mapping values"):
    # Assuming the correct dimensions are 'y' and 'x' (replace these with actual names from data.coords)

        B01_value = data.sel(x=lon, y=lat, band=1, method="nearest").values
        B01_values.append(B01_value)

        B02_value = data.sel(x=lon, y=lat, band=2, method="nearest").values
        B02_values.append(B02_value)

        B03_value = data.sel(x=lon, y=lat, band=1, method="nearest").values
        B03_values.append(B03_value)

        B04_value = data.sel(x=lon, y=lat, band=2, method="nearest").values
        B04_values.append(B04_value)

        B05_value = data.sel(x=lon, y=lat, band=1, method="nearest").values
        B05_values.append(B05_value)

        B06_value = data.sel(x=lon, y=lat, band=3, method="nearest").values
        B06_values.append(B06_value)

        B07_value = data.sel(x=lon, y=lat, band=3, method="nearest").values
        B07_values.append(B07_value)

        B08_value = data.sel(x=lon, y=lat, band=4, method="nearest").values
        B08_values.append(B08_value)

        B8A_value = data.sel(x=lon, y=lat, band=4, method="nearest").values
        B8A_values.append(B8A_value)

        B11_value = data.sel(x=lon, y=lat, band=4, method="nearest").values
        B11_values.append(B11_value)

        B12_value = data.sel(x=lon, y=lat, band=4, method="nearest").values
        B12_values.append(B12_value)

    # Create a DataFrame with the band values
    # Create a DataFrame to store the band values
    df = pd.DataFrame()
    df['SB01'] = B01_values
    df['SB02'] = B02_values
    df['SB03'] = B03_values
    df['SB04'] = B04_values
    df['SB06'] = B06_values
    df['SB07'] = B07_values
    df['SB08'] = B08_values
    df['SB8A'] = B8A_values
    df['SB11'] = B11_values
    df['SB12'] = B12_values

    return df


In [23]:
# Mapping satellite data with training data.
S2_data = map_satellite_data('datasets/S2_sample_all.tiff', 'datasets/data.csv')
S2_sub = map_satellite_data('datasets/S2_sample_all.tiff', 'datasets/template.csv')

Mapping values: 100%|██████████| 1040/1040 [00:19<00:00, 54.40it/s]


In [24]:
display(S2_data.head())
display(S2_sub.head())

,SB01,SB02,SB03,SB04,SB06,SB07,SB08,SB8A,SB11,SB12
0,846.0,1042.0,846.0,1042.0,1036.0,1036.0,1036.0,1036.0,1036.0,1036.0
1,846.0,1042.0,846.0,1042.0,1036.0,1036.0,1036.0,1036.0,1036.0,1036.0
2,846.0,583.0,846.0,583.0,818.0,818.0,709.0,709.0,709.0,709.0
3,846.0,581.0,846.0,581.0,733.0,733.0,657.0,657.0,657.0,657.0
4,846.0,655.0,846.0,655.0,744.0,744.0,745.0,745.0,745.0,745.0


,SB01,SB02,SB03,SB04,SB06,SB07,SB08,SB8A,SB11,SB12
0,811.0,459.0,811.0,459.0,617.0,617.0,432.0,432.0,432.0,432.0
1,1208.0,562.0,1208.0,562.0,731.0,731.0,647.0,647.0,647.0,647.0
2,899.0,955.0,899.0,955.0,1052.0,1052.0,1188.0,1188.0,1188.0,1188.0
3,1193.0,1132.0,1193.0,1132.0,1364.0,1364.0,1512.0,1512.0,1512.0,1512.0
4,1097.0,1506.0,1097.0,1506.0,1642.0,1642.0,1688.0,1688.0,1688.0,1688.0


In [25]:
# Combining ground data and final data into a single dataset.
uhi_data = pd.concat([data_df,S2_data], axis=1)
display(uhi_data)

uhi_sub = pd.concat([sub_df,S2_sub], axis=1)
display(uhi_sub)

,Longitude,Latitude,Datetime,UHI Index,Region,nearest_building_distance,building_density,avg_building_age,avg_building_elevation,avg_building_height,...,SB01,SB02,SB03,SB04,SB06,SB07,SB08,SB8A,SB11,SB12
0,-73.909167,40.813107,2021-07-24 15:53:00,1.030289,Bronx,28.702934,10957,76.409679,59.732250,39.734542,...,846.0,1042.0,846.0,1042.0,1036.0,1036.0,1036.0,1036.0,1036.0,1036.0
1,-73.909187,40.813045,2021-07-24 15:53:00,1.030289,Bronx,27.747556,10942,76.425754,59.670272,39.761382,...,846.0,1042.0,846.0,1042.0,1036.0,1036.0,1036.0,1036.0,1036.0,1036.0
2,-73.909215,40.812978,2021-07-24 15:53:00,1.023798,Bronx,26.465094,10923,76.404365,59.600471,39.763419,...,846.0,583.0,846.0,583.0,818.0,818.0,709.0,709.0,709.0,709.0
3,-73.909242,40.812908,2021-07-24 15:53:00,1.023798,Bronx,25.873093,10901,76.447440,59.550175,39.775245,...,846.0,581.0,846.0,581.0,733.0,733.0,657.0,657.0,657.0,657.0
4,-73.909257,40.812845,2021-07-24 15:53:00,1.021634,Bronx,26.359730,10883,76.461466,59.513618,39.775971,...,846.0,655.0,846.0,655.0,744.0,744.0,745.0,745.0,745.0,745.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11224,-73.957050,40.790333,2021-07-24 15:57:00,0.972470,Manhattan,161.437295,12349,101.955419,58.208364,73.181264,...,481.0,473.0,481.0,473.0,708.0,708.0,528.0,528.0,528.0,528.0
11225,-73.957063,40.790308,2021-07-24 15:57:00,0.972470,Manhattan,161.166533,12344,101.963042,58.221419,73.191028,...,481.0,540.0,481.0,540.0,742.0,742.0,610.0,610.0,610.0,610.0
11226,-73.957093,40.790270,2021-07-24 15:57:00,0.981124,Manhattan,161.138005,12335,101.964168,58.224196,73.228970,...,481.0,540.0,481.0,540.0,742.0,742.0,610.0,610.0,610.0,610.0
11227,-73.957112,40.790253,2021-07-24 15:59:00,0.981245,Manhattan,161.353627,12336,101.966226,58.248520,73.240985,...,481.0,540.0,481.0,540.0,742.0,742.0,610.0,610.0,610.0,610.0


,Longitude,Latitude,UHI Index,Region,nearest_building_distance,building_density,avg_building_age,avg_building_elevation,avg_building_height,avg_height_above_ground,...,SB01,SB02,SB03,SB04,SB06,SB07,SB08,SB8A,SB11,SB12
0,-73.971665,40.788763,NaN,Manhattan,24.223804,9480,106.813675,72.826956,95.329320,22.502364,...,811.0,459.0,811.0,459.0,617.0,617.0,432.0,432.0,432.0,432.0
1,-73.971928,40.788875,NaN,Manhattan,18.437818,9425,106.924788,72.876599,95.641470,22.764872,...,1208.0,562.0,1208.0,562.0,731.0,731.0,647.0,647.0,647.0,647.0
2,-73.967080,40.789080,NaN,Manhattan,45.864314,10229,106.268013,70.930495,88.069186,17.138691,...,899.0,955.0,899.0,955.0,1052.0,1052.0,1188.0,1188.0,1188.0,1188.0
3,-73.972550,40.789082,NaN,Manhattan,4.973082,9363,107.083811,72.799308,96.101098,23.301791,...,1193.0,1132.0,1193.0,1132.0,1364.0,1364.0,1512.0,1512.0,1512.0,1512.0
4,-73.969697,40.787953,NaN,Manhattan,39.096254,9745,106.378912,72.361802,92.955090,20.593288,...,1097.0,1506.0,1097.0,1506.0,1642.0,1642.0,1688.0,1688.0,1688.0,1688.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1035,-73.919388,40.813803,NaN,Bronx,27.991765,8911,76.912668,56.716524,41.501136,-15.215388,...,1474.0,1086.0,1474.0,1086.0,1382.0,1382.0,1474.0,1474.0,1474.0,1474.0
1036,-73.931033,40.833178,NaN,Bronx,25.217523,8922,91.454107,71.470845,52.338347,-19.132499,...,1014.0,548.0,1014.0,548.0,766.0,766.0,797.0,797.0,797.0,797.0
1037,-73.934647,40.854542,NaN,Manhattan,19.660984,5461,93.230841,98.281975,55.533750,-42.748225,...,917.0,1184.0,917.0,1184.0,1462.0,1462.0,1538.0,1538.0,1538.0,1538.0
1038,-73.917223,40.815413,NaN,Bronx,34.562437,9502,76.978893,57.478799,41.349281,-16.129518,...,1890.0,1066.0,1890.0,1066.0,1244.0,1244.0,1368.0,1368.0,1368.0,1368.0


# Combine Landsat Information

In [ ]:
# def map_landsat_data(tiff_path, csv_path):
#     # Open the Landsat GeoTIFF data
#     with rasterio.open(tiff_path) as src:
#         tiff_crs = src.crs
#         num_bands = src.count  # Get the total number of bands in the TIFF
#         bands = {}

#         # Read all available bands (using num_bands dynamically)
#         for band_num in range(1, num_bands + 1):
#             bands[band_num] = src.read(band_num)

#     # Read the CSV with coordinates
#     df = pd.read_csv(csv_path)
#     latitudes = df['Latitude'].values
#     longitudes = df['Longitude'].values

#     # Define the CRS transformation (WGS84 to the CRS of the GeoTIFF)
#     proj_wgs84 = Proj(init='epsg:4326')  # EPSG:4326 is the common lat/long CRS
#     proj_tiff = Proj(tiff_crs.to_string())  # CRS of the Landsat image
#     transformer = Transformer.from_proj(proj_wgs84, proj_tiff)

#     # Prepare lists for storing band values
#     band_values = {f'LB{band_num:02}': [] for band_num in range(1, num_bands + 1)}

#     # Get the dimensions of the raster for bounds checking
#     raster_height, raster_width = bands[1].shape

#     # Iterate over the latitudes and longitudes to extract corresponding band values
#     for lat, lon in tqdm(zip(latitudes, longitudes), total=len(latitudes), desc="Mapping Landsat values"):
#         # Convert latitude/longitude to the GeoTIFF's CRS
#         x, y = transformer.transform(lon, lat)

#         # Round to nearest integer index
#         x = int(np.round(x))
#         y = int(np.round(y))

#         # Ensure that the indices are within the bounds of the raster
#         if 0 <= x < raster_width and 0 <= y < raster_height:
#             for band_num in range(1, num_bands + 1):
#                 band_value = bands[band_num][y, x]  # Get the value for the corresponding band
#                 band_values[f'LB{band_num:02}'].append(band_value)
#         else:
#             # Handle the case where the coordinate is outside the image bounds
#             for band_num in range(1, num_bands + 1):
#                 band_values[f'LB{band_num:02}'].append(np.nan)  # Add NaN if the coordinate is out of bounds

#     # Create a DataFrame to store the band values
#     df_out = pd.DataFrame(band_values)

#     return df_out

In [ ]:
# # Example of how to use the function:
# landsat_data = map_landsat_data('datasets/Landsat_LST.tiff', 'datasets/data.csv')
# landsat_sub = map_landsat_data('datasets/Landsat_LST.tiff', 'datasets/template.csv')

# display(landsat_data.head())
# display(landsat_sub.head())


# Save final combined datasets

In [26]:
print(uhi_data.columns)
print(uhi_sub.columns)

Index(['Longitude', 'Latitude', 'Datetime', 'UHI Index', 'Region',
       'nearest_building_distance', 'building_density', 'avg_building_age',
       'avg_building_elevation', 'avg_building_height',
       'avg_height_above_ground', 'Air Temp at Surface', 'Relative Humidity',
       'Avg Wind Speed', 'Wind Direction', 'Solar Flux', 'SB01', 'SB02',
       'SB03', 'SB04', 'SB06', 'SB07', 'SB08', 'SB8A', 'SB11', 'SB12'],
      dtype='object')
Index(['Longitude', 'Latitude', 'UHI Index', 'Region',
       'nearest_building_distance', 'building_density', 'avg_building_age',
       'avg_building_elevation', 'avg_building_height',
       'avg_height_above_ground', 'Datetime', 'Air Temp at Surface',
       'Relative Humidity', 'Avg Wind Speed', 'Wind Direction', 'Solar Flux',
       'SB01', 'SB02', 'SB03', 'SB04', 'SB06', 'SB07', 'SB08', 'SB8A', 'SB11',
       'SB12'],
      dtype='object')


In [27]:
display(uhi_data)
display(uhi_sub)

,Longitude,Latitude,Datetime,UHI Index,Region,nearest_building_distance,building_density,avg_building_age,avg_building_elevation,avg_building_height,...,SB01,SB02,SB03,SB04,SB06,SB07,SB08,SB8A,SB11,SB12
0,-73.909167,40.813107,2021-07-24 15:53:00,1.030289,Bronx,28.702934,10957,76.409679,59.732250,39.734542,...,846.0,1042.0,846.0,1042.0,1036.0,1036.0,1036.0,1036.0,1036.0,1036.0
1,-73.909187,40.813045,2021-07-24 15:53:00,1.030289,Bronx,27.747556,10942,76.425754,59.670272,39.761382,...,846.0,1042.0,846.0,1042.0,1036.0,1036.0,1036.0,1036.0,1036.0,1036.0
2,-73.909215,40.812978,2021-07-24 15:53:00,1.023798,Bronx,26.465094,10923,76.404365,59.600471,39.763419,...,846.0,583.0,846.0,583.0,818.0,818.0,709.0,709.0,709.0,709.0
3,-73.909242,40.812908,2021-07-24 15:53:00,1.023798,Bronx,25.873093,10901,76.447440,59.550175,39.775245,...,846.0,581.0,846.0,581.0,733.0,733.0,657.0,657.0,657.0,657.0
4,-73.909257,40.812845,2021-07-24 15:53:00,1.021634,Bronx,26.359730,10883,76.461466,59.513618,39.775971,...,846.0,655.0,846.0,655.0,744.0,744.0,745.0,745.0,745.0,745.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11224,-73.957050,40.790333,2021-07-24 15:57:00,0.972470,Manhattan,161.437295,12349,101.955419,58.208364,73.181264,...,481.0,473.0,481.0,473.0,708.0,708.0,528.0,528.0,528.0,528.0
11225,-73.957063,40.790308,2021-07-24 15:57:00,0.972470,Manhattan,161.166533,12344,101.963042,58.221419,73.191028,...,481.0,540.0,481.0,540.0,742.0,742.0,610.0,610.0,610.0,610.0
11226,-73.957093,40.790270,2021-07-24 15:57:00,0.981124,Manhattan,161.138005,12335,101.964168,58.224196,73.228970,...,481.0,540.0,481.0,540.0,742.0,742.0,610.0,610.0,610.0,610.0
11227,-73.957112,40.790253,2021-07-24 15:59:00,0.981245,Manhattan,161.353627,12336,101.966226,58.248520,73.240985,...,481.0,540.0,481.0,540.0,742.0,742.0,610.0,610.0,610.0,610.0


,Longitude,Latitude,UHI Index,Region,nearest_building_distance,building_density,avg_building_age,avg_building_elevation,avg_building_height,avg_height_above_ground,...,SB01,SB02,SB03,SB04,SB06,SB07,SB08,SB8A,SB11,SB12
0,-73.971665,40.788763,NaN,Manhattan,24.223804,9480,106.813675,72.826956,95.329320,22.502364,...,811.0,459.0,811.0,459.0,617.0,617.0,432.0,432.0,432.0,432.0
1,-73.971928,40.788875,NaN,Manhattan,18.437818,9425,106.924788,72.876599,95.641470,22.764872,...,1208.0,562.0,1208.0,562.0,731.0,731.0,647.0,647.0,647.0,647.0
2,-73.967080,40.789080,NaN,Manhattan,45.864314,10229,106.268013,70.930495,88.069186,17.138691,...,899.0,955.0,899.0,955.0,1052.0,1052.0,1188.0,1188.0,1188.0,1188.0
3,-73.972550,40.789082,NaN,Manhattan,4.973082,9363,107.083811,72.799308,96.101098,23.301791,...,1193.0,1132.0,1193.0,1132.0,1364.0,1364.0,1512.0,1512.0,1512.0,1512.0
4,-73.969697,40.787953,NaN,Manhattan,39.096254,9745,106.378912,72.361802,92.955090,20.593288,...,1097.0,1506.0,1097.0,1506.0,1642.0,1642.0,1688.0,1688.0,1688.0,1688.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1035,-73.919388,40.813803,NaN,Bronx,27.991765,8911,76.912668,56.716524,41.501136,-15.215388,...,1474.0,1086.0,1474.0,1086.0,1382.0,1382.0,1474.0,1474.0,1474.0,1474.0
1036,-73.931033,40.833178,NaN,Bronx,25.217523,8922,91.454107,71.470845,52.338347,-19.132499,...,1014.0,548.0,1014.0,548.0,766.0,766.0,797.0,797.0,797.0,797.0
1037,-73.934647,40.854542,NaN,Manhattan,19.660984,5461,93.230841,98.281975,55.533750,-42.748225,...,917.0,1184.0,917.0,1184.0,1462.0,1462.0,1538.0,1538.0,1538.0,1538.0
1038,-73.917223,40.815413,NaN,Bronx,34.562437,9502,76.978893,57.478799,41.349281,-16.129518,...,1890.0,1066.0,1890.0,1066.0,1244.0,1244.0,1368.0,1368.0,1368.0,1368.0


In [28]:
uhi_data.to_csv('data_s2.csv', index=False)
uhi_sub.to_csv('sub_s2.csv', index=False)